In [3]:
import numpy as np
import pandas as pd
from typing import Dict, Tuple
from scipy import stats
from IPython.display import Image
from IPython.display import Image
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.datasets import load_iris, load_boston
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
#from surprise import SVD, Dataset, Reader
#from surprise.model_selection import PredefinedKFold
from collections import defaultdict
#from surprise.accuracy import rmse
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
%matplotlib inline 
sns.set(style="ticks")

In [19]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [20]:
data = pd.read_csv('/content/drive/MyDrive/books.csv', sep=";", 
                    error_bad_lines=False, encoding="latin-1")
data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [21]:
data.shape

(271360, 8)

In [22]:
data.drop_duplicates(['Year-Of-Publication'])

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
6,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...
...,...,...,...,...,...,...,...,...
245607,0781268001,"Hugh Wynne, Free Quaker (2 Volumes (BCL1-PS Am...",Silas Weir Mitchell,1897,Reprint Services Corp,http://images.amazon.com/images/P/0781268001.0...,http://images.amazon.com/images/P/0781268001.0...,http://images.amazon.com/images/P/0781268001.0...
246842,0380000059,CLOUT,D. GIBBONS,2024,Avon,http://images.amazon.com/images/P/0380000059.0...,http://images.amazon.com/images/P/0380000059.0...,http://images.amazon.com/images/P/0380000059.0...
253750,964442011X,Tasht-i khun,IsmaÂ°il Fasih,1376,Nashr-i Alburz,http://images.amazon.com/images/P/964442011X.0...,http://images.amazon.com/images/P/964442011X.0...,http://images.amazon.com/images/P/964442011X.0...
253831,0460008005,Holinshed's Chronicle,Raphael Holinshed,1926,J m Dent &amp; Sons Ltd,http://images.amazon.com/images/P/0460008005.0...,http://images.amazon.com/images/P/0460008005.0...,http://images.amazon.com/images/P/0460008005.0...


In [23]:
overview_data_=data[data['Year-Of-Publication'].notnull()]

In [24]:
Plot_data=overview_data_[overview_data_['Publisher'].notnull()]

In [25]:
title=Plot_data['Year-Of-Publication'].values

In [26]:
title[0:5]

array([2002, 2001, 1991, 1999, 1999], dtype=object)

In [27]:
Plot=Plot_data['Publisher'].values
Plot[0:5]

array(['Oxford University Press', 'HarperFlamingo Canada',
       'HarperPerennial', 'Farrar Straus Giroux',
       'W. W. Norton &amp; Company'], dtype=object)

In [28]:
index=np.arange(len(Plot))
index

array([     0,      1,      2, ..., 271355, 271356, 271357])

In [29]:
%%time
tfidf = TfidfVectorizer()
Plot_matrix = tfidf.fit_transform(Plot)
Plot_matrix

CPU times: user 2.14 s, sys: 13.9 ms, total: 2.16 s
Wall time: 4.32 s


In [30]:
Plot_matrix

<271358x10884 sparse matrix of type '<class 'numpy.float64'>'
	with 628138 stored elements in Compressed Sparse Row format>

In [31]:
class SimpleKNNRecommender:
    
    def __init__(self, X_matrix, X_ids, X_title, X_overview):
        """
        Входные параметры:
        X_matrix - обучающая выборка (матрица объект-признак)
        X_ids - массив идентификаторов объектов
        X_title - массив названий объектов
        X_overview - массив описаний объектов
        """
        #Сохраняем параметры в переменных объекта
        self._X_matrix = X_matrix
        self.df = pd.DataFrame(
            {'id': pd.Series(X_ids, dtype='int'),
            'title': pd.Series(X_title, dtype='str'),
            'overview': pd.Series(X_overview, dtype='str'),
            'dist': pd.Series([], dtype='float')})
            
            
    def recommend_for_single_object(self, K: int, \
                X_matrix_object, cos_flag = True, manh_flag = False):
        """
        Метод формирования рекомендаций для одного объекта.
        Входные параметры:
        K - количество рекомендуемых соседей 
        X_matrix_object - строка матрицы объект-признак, соответствующая объекту
        cos_flag - флаг вычисления косинусного расстояния
        manh_flag - флаг вычисления манхэттэнского расстояния
        Возвращаемое значение: K найденных соседей
        """
        
        scale = 1000000
        # Вычисляем косинусную близость
        if cos_flag:
            dist = cosine_similarity(self._X_matrix, X_matrix_object)
            self.df['dist'] = dist * scale
            res = self.df.sort_values(by='dist', ascending=False)
            # Не учитываем рекомендации с единичным расстоянием,
            # так как это искомый объект
            res = res[res['dist'] < scale]
        
        else:
            if manh_flag:
                dist = manhattan_distances(self._X_matrix, X_matrix_object)
            else:
                dist = euclidean_distances(self._X_matrix, X_matrix_object)
            self.df['dist'] = dist * scale
            res = self.df.sort_values(by='dist', ascending=True)
            # Не учитываем рекомендации с единичным расстоянием,
            # так как это искомый объект
            res = res[res['dist'] > 0.0]            
        
        # Оставляем К первых рекомендаций
        res = res.head(K)
        return res

In [32]:
demo_id=1100
print(Plot[demo_id])

Bantam Books


In [33]:
demo_matrix = Plot_matrix[demo_id]

In [34]:
skr1 = SimpleKNNRecommender(Plot_matrix,index,title,Plot)


In [35]:
rec1 = skr1.recommend_for_single_object(15, demo_matrix)
rec1

,id,title,overview,dist
102175,102175,1989,Bantam,875780.432157
46550,46550,1989,Bantam,875780.432157
164082,164082,2000,Bantam,875780.432157
21563,21563,1996,Bantam,875780.432157
90040,90040,1990,Bantam,875780.432157
8436,8436,1991,Bantam,875780.432157
233855,233855,1990,Bantam,875780.432157
4871,4871,1998,Bantam,875780.432157
56277,56277,1997,Bantam,875780.432157
102042,102042,1987,Bantam,875780.432157


In [36]:
rec2 = skr1.recommend_for_single_object(15,demo_matrix, cos_flag = False)
rec2

,id,title,overview,dist
39404,39404,2003,Bantam,498436.691753
86909,86909,1991,Bantam,498436.691753
75159,75159,1993,Bantam,498436.691753
69305,69305,1988,Bantam,498436.691753
5166,5166,1990,Bantam,498436.691753
30109,30109,1993,Bantam,498436.691753
62679,62679,2003,Bantam,498436.691753
29096,29096,1990,Bantam,498436.691753
126392,126392,2004,Bantam,498436.691753
38563,38563,2004,Bantam,498436.691753


In [37]:
rec3 = skr1.recommend_for_single_object(15,demo_matrix,cos_flag = False, manh_flag = True)
rec3

,id,title,overview,dist
7798,7798,2004,Bantam,606929.24731
51597,51597,1993,Bantam,606929.24731
114736,114736,1993,Bantam,606929.24731
105870,105870,1999,Bantam,606929.24731
34266,34266,2000,Bantam,606929.24731
14592,14592,1983,Bantam,606929.24731
129952,129952,1995,Bantam,606929.24731
114715,114715,1998,Bantam,606929.24731
10586,10586,1991,Bantam,606929.24731
86187,86187,1987,Bantam,606929.24731


In [38]:
data2=data.iloc[:15000,:]

In [40]:
len(data2['Book-Author'].unique())

8218

In [43]:
len(data2['Publisher'].unique())

2285

In [46]:
def create_utility_matrix(data):
    itemField = 'Book-Author'
    userField = 'Publisher'
    valueField = 'Year-Of-Publication'  
    
    userList = data[userField].tolist()
    itemList = data[itemField].tolist()
    valueList = data[valueField].tolist()    
    
    users = list(set(userList))
    items = list(set(itemList))    
    
    users_index = {users[i]: i for i in range(len(users))}    
    pd_dict = {item: [0.0 for i in range(len(users))] for item in items}    
    
    for i in range(0,data.shape[0]):
        item = itemList[i]
        user = userList[i]
        value = valueList[i]    
        pd_dict[item][users_index[user]] = value    
    
    X = pd.DataFrame(pd_dict)
    X.index = users
        
    itemcols = list(X.columns)
    items_index = {itemcols[i]: i for i in range(len(itemcols))}
    
    return X, users_index, items_index

In [47]:
%%time
user_item_matrix, users_index, items_index = create_utility_matrix(data2)

CPU times: user 3.85 s, sys: 506 ms, total: 4.36 s
Wall time: 4.06 s


In [48]:
user_item_matrix

,Anne Lindsay,A. Wimschneider,Alison Wearing,Mick Jackson,Andrew Greeley,Paul Erdman,B. K. S. Iyengar,Linda Gordon,Lorraine Anderson,Micah Sparks,...,Sharon A. Feaster,Ahne,Mick Middles,Amanda Quick,Paul Gambaccini,Judy Alter,Carolyn Keene,Phil Farrand,John O'Donohue,Jody Seay
Peachtree Publishers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Macmillan Pub Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Brimax Books Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U.S. Games Systems,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
USCCB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HarperFestival,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kodansha America,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Beacon Press,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Editores Mexicanos Unidos, S.A.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
user_item_matrix__test = user_item_matrix.loc[['HarperPerennial']]
user_item_matrix__test

,Anne Lindsay,A. Wimschneider,Alison Wearing,Mick Jackson,Andrew Greeley,Paul Erdman,B. K. S. Iyengar,Linda Gordon,Lorraine Anderson,Micah Sparks,...,Sharon A. Feaster,Ahne,Mick Middles,Amanda Quick,Paul Gambaccini,Judy Alter,Carolyn Keene,Phil Farrand,John O'Donohue,Jody Seay
HarperPerennial,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
Title = np.delete(data2['Publisher'].unique(), 0)
Title = np.delete(Title, 1)
Title

array(['HarperFlamingo Canada', 'Farrar Straus Giroux',
       'W. W. Norton &amp; Company', ..., 'S.P.I. Books',
       'Open Window Creations', 'Smithmark Publishing'], dtype=object)

In [53]:
user_item_matrix__train = user_item_matrix.loc[Title]
user_item_matrix__train = user_item_matrix__train.astype(np.float64)
user_item_matrix__train

,Anne Lindsay,A. Wimschneider,Alison Wearing,Mick Jackson,Andrew Greeley,Paul Erdman,B. K. S. Iyengar,Linda Gordon,Lorraine Anderson,Micah Sparks,...,Sharon A. Feaster,Ahne,Mick Middles,Amanda Quick,Paul Gambaccini,Judy Alter,Carolyn Keene,Phil Farrand,John O'Donohue,Jody Seay
HarperFlamingo Canada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Farrar Straus Giroux,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
W. W. Norton &amp; Company,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Putnam Pub Group,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Berkley Publishing Group,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1998.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Dietz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Econ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S.P.I. Books,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Open Window Creations,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
%%time
U, S, VT = np.linalg.svd(user_item_matrix__train.T)
V = VT.T

CPU times: user 1min 12s, sys: 4.77 s, total: 1min 17s
Wall time: 43.7 s


In [55]:
U.shape

(8218, 8218)

In [56]:
V.shape

(2283, 2283)

In [57]:
S.shape

(2283,)

In [58]:
Sigma = np.diag(S)
Sigma.shape

(2283, 2283)

In [59]:
Sigma

array([[3.39074652e+04, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.05169201e+04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.87580253e+04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        6.04184123e-14, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 3.34681109e-14, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 2.08645683e-14]])

In [60]:
r=3
Ur = U[:, :r]
Sr = Sigma[:r, :r]
Vr = V[:, :r]
# Матрица соотношения между новым дегустатором и латентными факторами
test_user = np.mat(user_item_matrix__test.values)
test_user = test_user.astype(np.float64)
test_user.shape, test_user

((1, 8218), matrix([[0., 0., 0., ..., 0., 0., 0.]]))

In [61]:
tmp = test_user * Ur * np.linalg.inv(Sr)
tmp

matrix([[0., 0., 0.]])

In [62]:
test_user_result = np.array([tmp[0,0], tmp[0,1], tmp[0,2]])
test_user_result

array([0., 0., 0.])

In [63]:
cos_sim = cosine_similarity(Vr, test_user_result.reshape(1, -1))
cos_sim[:10]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [64]:
cos_sim_list = cos_sim.reshape(-1, cos_sim.shape[0])[0]
cos_sim_list[:10]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [65]:
recommended_user_id = np.argsort(-cos_sim_list)[0]
recommended_user_id

0

In [66]:
test_user

matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [67]:
movieId_list = list(user_item_matrix.columns)
def film_name_by_movieid(ind):
    try:
        movieId = movieId_list[ind]
        flt_links = data2[data2['Book-Author'] == movieId]
        tmdbId = int(flt_links['Publisher	'].values[0])
        md_links = data2[data2['id'] == tmdbId]
        res = md_links['title'].values[0]
        return res
    except:
        return '' 

In [68]:
i=1
for idx, item in enumerate(np.ndarray.flatten(np.array(test_user))):
    if item > 0:
        film_title = film_name_by_movieid(idx)
        print('{} - {} - {}'.format(idx, Title[i], item))
        if i==10000:
            break
        else:
            i+=1

4284 - Farrar Straus Giroux - 1991.0
